# A4 Implement a RESTful service

Due: before **28 November 11:59**

In this assignment you will implement a simple database application with a REST interface. You'll use the Python [Bottle](https://bottlepy.org/docs/dev/tutorial.html) framework to handle the routing of requests to your code. I'll show you how it is done with an example below.

We're running the server inside this notebook. Of course in the real world you wouldn't do it like this but the code would be nearly the same. Look at the examples in the [REST lecture notes](https://wwwx.cs.unc.edu/Courses/comp521-f18/media/CH7-InternetReST.html) for details. You don't have to do any of that for this assignment.

A bit of explanation about how this is working. You are writing functions that are bound to **routes** by the Bottle framework. The **decorator** above the function definition describes the binding. It looks like magic but under the
covers they are simply building a table that maps strings in URLs to function calls. A simple route might look like this:

```python
@app.route('/foo')        # this is the decorator describing the route
def index():              # this is the function to call
    return {'r': 'Hello'} # return an object with key r and value "Hello"
```

When I am testing your code, I call a function I wrote named `getJson`. `getJson('foo')` uses the Python standard library to submit an HTTP request to `http://localhost:8080/foo`. The Bottle server (running in a background thread) receives the request, finds your function, calls it, and then packages whatever the function returns as a HTTP response. `getJson` gets that response, interprets the response code and decodes the JSON, if any. So, it may look like magic but it is all pretty standard web stuff. The one weird thing here is that the **server** (your code) and the **client** (my test code) are running in the same program. You can see the magic in the `server.py` file included with the assignment.

If you would like to see your server working, simply open another tab in your browser and enter a URL like http://localhost:8080/classes to see the list of classes. I use the [JSONview extension for Chrome](https://chrome.google.com/webstore/detail/jsonview/chklaanhfefbnpoihckbnefhakgolnmc?hl=en) to nicely format the JSON for me but that isn't necessary for this assignment.

## DB Schema

```sql
CREATE TABLE student (
    sid INTEGER PRIMARY KEY,
    fname TEXT,
    lname TEXT)

CREATE TABLE course (
    cid INTEGER PRIMARY KEY,
    name TEXT,
    number TEXT,
    department TEXT,
    credit_hours INTEGER)

CREATE TABLE enrolled_in (
    sid INTEGER,
    cid INTEGER,
    FOREIGN KEY(sid) REFERENCES student,
    FOREIGN KEY(cid) REFERENCES course,
    UNIQUE (sid, cid))
```

## Setup

Some text with a pink background will appear below this next cell. It should read:

```
Bottle v0.13-dev server starting up (using MyWSGIRefServer())...
Listening on http://127.0.0.1:8080/
Hit Ctrl-C to quit.
```

In [1]:
import bottle
import sqlite3

# I have a hacked this so that the server runs in a background thread allowing 
# you go keep working without restarting it. Feel free to look at the code but
# don't change it.
from server import app, root, getJson, makeURL

import comp521
check, report = comp521.start('A4')

# some text should appear below with a pink background. It looks like an error 
# but the notebook is showing it in pink because it came out on stderr.

Bottle v0.13-dev server starting up (using MyWSGIRefServer())...
Listening on http://127.0.0.1:8080/
Hit Ctrl-C to quit.



## Setup (optional)
**This next cell is completely optional.** [James Jushchuk](mailto:jushchuk@cs.unc.edu) (one of the TA's) is doing research on providing help to students based on the work of other students. For this research he need student data. By agreeing to help with the research your work would be logged locally in a file called A4_log.json in the same folder as A4. You would then email this file to James (jushchuk@cs.unc.edu) upon completion of A4.

**This is completely voluntary**, but if you would like to participate, please read [this consent form](https://docs.google.com/document/d/1pgwci_3N1F8DLJy58SNhJ5cBNqg-lxr2AptYkAcWkes/edit?usp=sharing), change the IRB_CONSENT to True, and uncomment the lines below.

If you ever want to stop participating you can simply restart the notebook, re-comment the lines, and delete A4_log.json. 

This will only log A4.ipynb and only when A4.ipynb is running. Nothing else is logged. Feel free to look at the logger.py to see what is logged but don't change it.

In [2]:
#import logger
#IRB_consent = False
#logger.start(IRB_consent)
#%autosave 1

## Helpers

### Reset DB

This cell defines a function that will reset the DB back to the initial state. This will prevent you from going insane because you can't get the correct answer with a database that is different from mine. Of course you wouldn't do this in the real world.

In [3]:
# don't change this

import shutil
import os
import stat

dbName = 'A4.db'

def resetDB():
    """Restore the db to the original state"""
    os.chmod('A4Save.db', stat.S_IRUSR)
    shutil.copyfile('A4Save.db', dbName)
    
resetDB()

### Connect to DB

Use this function inside your handler to connect to the DB. It will take care of closing it for you when the request is completed.

In [4]:
# don't change this

def connect(name):
    '''Return a db connection that will close and commit automatically'''
    conn = sqlite3.connect(name)
    conn.row_factory = sqlite3.Row
    return conn

### makeURL Helper function

Use this to create the URLs to hand back in the JSON. You just call it like

`makeURL('students', 123)`

and it will create a URL in the right format to return.

# Example

I'll complete this one for you in entirety as an example of what you need to do. Pay close attention to the structure.

### Q0. GET /students List students

When a GET request is received with URL `/students` you should return a JSON object with a single key `students` and value list of students in order by increasing `id`. Like this:

```
{ 'students': [
    {
        "lastName": "Bishop",
        "firstName": "Gary",
        "id": "http://localhost:8080/students/123"
    }
  ]
}
```

In [5]:
# I have completed this one. You do NOT need to change it

@app.route('/students')
def indexStudents():
    """List students"""
    # open the DB; commit and close when we exit the block
    with connect(dbName) as db:
        # run the sql query to get all the students
        cursor = db.execute('''select sid, lname, fname from student order by sid''')
        # reformat the data
        result = [ { "lastName": lname, "firstName": fname, "id": makeURL('students', sid) }
                   for sid, lname, fname in cursor ]
        # Bottle is going to magically convert this dictionary to JSON
        return { 'students': result }

check('Q0. List students', getJson('students'), points=0)

http response = 200
{'students': [{'firstName': 'Ashwin',
   'id': '/students/0',
   'lastName': 'Carter'},
  {'firstName': 'Alexandros', 'id': '/students/1', 'lastName': 'Velasquez'},
  {'firstName': 'Ainsley', 'id': '/students/2', 'lastName': 'Beach'},
  {'firstName': 'Kamile', 'id': '/students/3', 'lastName': 'Person'},
  {'firstName': 'Maciej', 'id': '/students/4', 'lastName': 'Buck'},
  {'firstName': 'Evalyn', 'id': '/students/5', 'lastName': 'Macfarlane'},
  {'firstName': 'Monty', 'id': '/students/6', 'lastName': 'Schroeder'},
  {'firstName': 'Kiri', 'id': '/students/7', 'lastName': 'Ryder'},
  ...]}
Q0. List students appears correct


127.0.0.1 - - [26/Nov/2018 19:21:02] "get /students HTTP/1.1" 200 10381


# Questions

### Q1. GET /students/123 Return specific student data

Return 404 if the given ID is invalid otherwise your result should look like this:
```
{
    "lastName": "Bishop",
    "firstName": "Gary",
    "id": "http://localhost:8080/students/3",
    "classes": [
        "http://localhost:8080/classes/1",
        "http://localhost:8080/classes/4"
    ]
}
```

Whenever you produce a list, order it by id.

You can produce a 404 response like this:

```python
return bottle.HTTPResponse(status=404)
```

In [7]:
@app.route('/students/<sid>')
def student(sid):
    """Return students info and classes"""
    with connect(dbName) as db:
        # write your code here
        cursor = db.execute('''select sid, lname, fname from student where sid = ?''',[sid])
        row = cursor.fetchone()
        cursor1 = db.execute('''select cid from enrolled_in where sid = ?''',[sid])
        classes = [makeURL('classes', i['cid']) for i in cursor1.fetchall()]
        
        if not row:
            return bottle.HTTPResponse(status=404)
        
        result = [{'lastName': row['lname'],
                  'firstName': row['fname'],
                  'id': makeURL('students', sid),
                  'classes': classes}]
        
        return {'lastName': row['lname'],
                  'firstName': row['fname'],
                  'id': makeURL('students', sid),
                  'classes': classes}


check('Q1. Student info', getJson('students', 1), points=5)

http response = 200
{'classes': ['/classes/1', '/classes/3', '/classes/25'],
 'firstName': 'Alexandros',
 'id': '/students/1',
 'lastName': 'Velasquez'}
Q1. Student info appears correct


127.0.0.1 - - [27/Nov/2018 19:58:58] "get /students/1 HTTP/1.1" 200 129


### Q2. DELETE /students/123

Delete the student and all of their enrollments. Produce a 404 if the student id is invalid. Produce a [204 response](https://tools.ietf.org/html/rfc7231#section-4.3.5) if successful.

In [13]:
@app.route('/students/<sid>', method="delete")
def deleteStudent(sid):
    """Delete a student and their enrollment"""
    with connect(dbName) as db:
        cursorA = db.execute('''select sid from student where sid = ?''',[sid])
        row = cursorA.fetchone()
    
        if row == None:
            return bottle.HTTPResponse(status=404)
    
        cursorC = db.execute('''delete from student where sid = ?''',[sid])
        cursorB = db.execute('''delete from enrolled_in where sid = ?''',[sid])
    
        return bottle.HTTPResponse(status=204)
    

resetDB()
# this one should succeed returning a 204
check('Q2. Student delete', getJson('students', 1, method='delete'))
# this one should fail returning 404
check('Q2. Student delete again', getJson('students', 1, method='delete'))

http response = 204
Q2. Student delete appears correct
http response = 404
Q2. Student delete again appears correct


127.0.0.1 - - [27/Nov/2018 20:07:03] "delete /students/1 HTTP/1.1" 204 0
127.0.0.1 - - [27/Nov/2018 20:07:03] "delete /students/1 HTTP/1.1" 404 0


### Q3. POST /students create a student

The POST body will be json like this:
```
{
    "lastName": "Bishop",
    "firstName": "Gary"
}
```
I have included the code below to extract the POST body and convert it from json to a Python dictionary ready
for you to access the values like `body['lastName']`.

Return a response like this:
```
{
    "lastName": "Bishop",
    "firstName": "Gary",
    "id": "http://localhost:8080/students/123"
}
```

The returned "id" should be the next id in the table. 

In [19]:
@app.route('/students', method='post')
def addStudent():
    """Insert a new student"""
    with connect(dbName) as db:
        body = bottle.request.json
        # write your code here
        cursor = db.execute('''insert into student (lname, fname) values (?,?)''', (body['lastName'], body['firstName']))
        row = cursor.fetchone()
        
        return {'lastName': body['lastName'],
                  'firstName': body['firstName'],
                  'id': makeURL('students', cursor.lastrowid)}

resetDB()
aStudent = { "lastName": "Bishop", "firstName": "Gary" }
check('Q3. Insert a new student', getJson('students', method='post', postData=aStudent))

http response = 200
{'firstName': 'Gary', 'id': '/students/150', 'lastName': 'Bishop'}
Q3. Insert a new student appears correct


127.0.0.1 - - [27/Nov/2018 21:00:53] "post /students HTTP/1.1" 200 66


### Q4. PUT /students/1 update a student

Update the name of an existing student. The PUT body will be json like this:
```
{
    "lastName": "Bishop",
    "firstName": "Gary"
}
```
Return a 204 response on success and 404 on failure.

In [34]:
@app.route('/students/<sid>', method='put')
def updateStudent(sid):
    """Update student info"""
    with connect(dbName) as db:
        body = bottle.request.json
        # write your code here
        cursor = db.execute('''select * from student where sid = ?''', [sid])
        row = cursor.fetchone()
        
        if row == None:
            return bottle.HTTPResponse(status=404)
        
        cursor = db.execute('''update student set lname = ?, fname = ? where sid = ?''', 
                            (body['lastName'], body['firstName'], sid))
        
        return bottle.HTTPResponse(status=204)

resetDB()
aStudent = { "lastName": "Bishop", "firstName": "Gary" }
check('Q4. Update a student', getJson('students', 1, method='put', postData=aStudent))
check('Q4. Update a bogus student', getJson('students', 999, method='put', postData=aStudent))

http response = 204
Q4. Update a student appears correct
http response = 404
Q4. Update a bogus student appears correct


127.0.0.1 - - [27/Nov/2018 21:36:34] "put /students/1 HTTP/1.1" 204 0
127.0.0.1 - - [27/Nov/2018 21:36:34] "put /students/999 HTTP/1.1" 404 0


### Q5. GET /classes Return list of classes

Return an object with single key `classes` and value list of objects in class id order like this:
```
{ "classes": [
    {
        "id": "http://localhost:8080/classes/123",
        "name": "Biology",
        "number": "345"
    }
  ]
}
```

In [57]:
@app.route('/classes')
def listClasses():
    """Return a list of classes"""
    # write your code here
    with connect(dbName) as db:
        cursor = db.execute('''select cid, name, number from course order by cid''')
    
        result = [{ "id": makeURL("classes", cid), "name": name, "number": number }
                   for cid, name, number in cursor ]
    
        return { 'classes': result }
    

resetDB()
check('Q5. List classes', getJson('classes'), points=5)

http response = 200
{'classes': [{'id': '/classes/0',
   'name': 'Principles of Biology',
   'number': '101'},
  {'id': '/classes/1',
   'name': 'Molecular Biology and Genetics',
   'number': '202'},
  {'id': '/classes/2', 'name': 'Biotransport', 'number': '315'},
  {'id': '/classes/3', 'name': 'Biomedical Solid Mechanics', 'number': '345'},
  {'id': '/classes/4',
   'name': 'Principles of Statistical Inference',
   'number': '600'},
  {'id': '/classes/5',
   'name': 'Introduction to Organic Chemistry I',
   'number': '261'},
  {'id': '/classes/6',
   'name': 'Introduction to Polymer Chemistry',
   'number': '420'},
  {'id': '/classes/7', 'name': 'Compliers', 'number': '520'},
  ...]}
Q5. List classes appears correct


127.0.0.1 - - [28/Nov/2018 12:22:28] "get /classes HTTP/1.1" 200 1956


### Q6. GET /classes/123 Return class info

Return info on a specific class; 404 if it doesn't exist. The result should look like this:

```json
{
    "id": "http://127.0.0.1:8080/classes/1",
    "name": "Molecular Biology and Genetics",
    "department": "BIOL",
    "number": 123,
    "credit": 3,
    "enrollment": 23
}
```

In [60]:
@app.route('/classes/<cid>')
def classInfo(cid):
    """Return info on a single class"""
    # write your code here
    with connect(dbName) as db:
        cursor = db.execute('''select cid, name, department, number, credit_hours from course where cid = ?''', [cid])
        cursor1 = len(db.execute('''select * from enrolled_in where cid = ?''', [cid]).fetchall())
        row = cursor.fetchone()
        if not row:
            return bottle.HTTPResponse(status=404)
    
        result = {"id": makeURL("classes", cid), "name": row["name"], "department": row["department"], 
                  "number": row["number"], "credit": row["credit_hours"], "enrollment":cursor1}
    
        return result

resetDB()
check('Q6. get class info', getJson('classes', 1), points=5)
check('Q6. get bogus class info', getJson('classes', 999), points=5)

http response = 200
{'credit': 4,
 'department': 'BIOL',
 'enrollment': 23,
 'id': '/classes/1',
 'name': 'Molecular Biology and Genetics',
 'number': '202'}
Q6. get class info appears correct
http response = 404
Q6. get bogus class info appears correct


127.0.0.1 - - [28/Nov/2018 12:23:31] "get /classes/1 HTTP/1.1" 200 132
127.0.0.1 - - [28/Nov/2018 12:23:31] "get /classes/999 HTTP/1.1" 404 0


### Q8. PUT /classes/123/students/1  Add student 1 to class 123

A [PUT](https://tools.ietf.org/html/rfc7231#section-4.3.4) request to a URL like `/classes/<cid>/students<sid>` should add the student to the class. It should return status 204 if the student was added, 404 if the student or class id is invalid, or 409 if the student is already in the class.

In [87]:
@app.route('/classes/<cid>/students/<sid>', method="put")
def addStudentToClass(cid, sid):
    """Add student to class"""
    # write your code here
    with connect(dbName) as db:
        cursor = db.execute('''select sid from student where sid = ?''',[sid])
        row = cursor.fetchone()
        cursor1 = db.execute('''select cid from course where cid = ?''', [cid])
        row1 = cursor1.fetchone()
    
        if row == None or row1 == None:
            return bottle.HTTPResponse(status=404)

        check_duplicate = db.execute('''select * from enrolled_in where sid = ? and cid = ?''', [sid,cid]).fetchone()        
        if not check_duplicate:
            db.execute('''insert into enrolled_in (sid, cid) values (?,?)''', (row[0],row1[0]))
            return bottle.HTTPResponse(status=204)
        else:
            return bottle.HTTPResponse(status=409)
        

resetDB()
check('Q8. add class', getJson('classes', 2, 'students', 1, method="put"), points=5)
check('Q8. add class again', getJson('classes', 2, 'students', 1, method="put"), points=5)
check('Q8. add bogus class', getJson('classes', 999, 'students', 1, method="put"), points=5)
check('Q8. add bogus student', getJson('classes', 1, 'students', 999, method="put"), points=5)

http response = 204
Q8. add class appears correct
http response = 409
Q8. add class again appears correct
http response = 404
Q8. add bogus class appears correct
http response = 404
Q8. add bogus student appears correct


127.0.0.1 - - [28/Nov/2018 14:37:45] "put /classes/2/students/1 HTTP/1.1" 204 0
127.0.0.1 - - [28/Nov/2018 14:37:45] "put /classes/2/students/1 HTTP/1.1" 409 0
127.0.0.1 - - [28/Nov/2018 14:37:45] "put /classes/999/students/1 HTTP/1.1" 404 0
127.0.0.1 - - [28/Nov/2018 14:37:45] "put /classes/1/students/999 HTTP/1.1" 404 0


### Q10. DELETE /classes/123/1  Drop student 1 from class 123

A [DELETE](https://tools.ietf.org/html/rfc7231#section-4.3.5) request to a URL like `/classes/<cid>/<sid>` should drop the student from the class. It should return status 204 if the student was dropped, or 404 if the student or class id is invalid.

In [88]:
@app.route('/classes/<cid>/students/<sid>', method="delete")
def dropStudentFromClass(cid, sid):
    """Drop student from class"""
    # write your code here
    with connect(dbName) as db:
        cursor = db.execute('''select sid from student where sid = ?''',[sid])
        row = cursor.fetchone()
        cursor1 = db.execute('''select cid from course where cid = ?''', [cid])
        row1 = cursor1.fetchone()
        cursor2 = db.execute('''select s.fname from student s, enrolled_in e, course c where e.sid = s.sid and c.cid = e.cid
                            and e.sid = ?''', [sid])
        row2 = cursor2.fetchone()
    
        if row == None or row1 == None or row2 == None:
            return bottle.HTTPResponse(status=404)
        else:
            db.execute('''delete from enrolled_in where sid = ?''', [sid])
            return bottle.HTTPResponse(status=204)
    

resetDB()

# this should succeed
check('Q10. drop class', getJson('classes', 1, 'students', 1, method="delete"), points=5)
# this should fail because the student is no longer in the class
check('Q10. drop class again', getJson('classes', 1, 'students', 1, method="delete"), points=5)
# this should fail because the class doesn't exist
check('Q10. drop bogus class', getJson('classes', 999, 'students', 1, method="delete"), points=5)
# this should fail because the student doesn't exist
check('Q10. drop bogus student', getJson('classes', 1, 'students', 999, method="delete"), points=5)

http response = 204
Q10. drop class appears correct
http response = 404
Q10. drop class again appears correct
http response = 404
Q10. drop bogus class appears correct
http response = 404
Q10. drop bogus student appears correct


127.0.0.1 - - [28/Nov/2018 14:40:46] "delete /classes/1/students/1 HTTP/1.1" 204 0
127.0.0.1 - - [28/Nov/2018 14:40:46] "delete /classes/1/students/1 HTTP/1.1" 404 0
127.0.0.1 - - [28/Nov/2018 14:40:46] "delete /classes/999/students/1 HTTP/1.1" 404 0
127.0.0.1 - - [28/Nov/2018 14:40:46] "delete /classes/1/students/999 HTTP/1.1" 404 0


# Submit your work

Enter your onyen and collaborators below and then run the cell. You'll get the usual submit button.

In [89]:
onyen = 'yuanming'
collaborators = [ 'rahuln', 'johnnyni', 'here' ]

# don't change this
report(onyen, collaborators)

  Collaborators: ['rahuln', 'johnnyni', 'here']
Report for yuanming
  18 of 18 appear correct, 60 of 60 points


## For optional logging study

If you are logging your work, but the size is too large to email, uncomment and run this next line.

In [ ]:
#logger.compress_log()